# Homework 4 - Kaggle Competition
## CSCI 5622 - Spring 2019
#Scott Scheraga
***
train-features.csv contains the features of the examples / cases, separated by commas, that you will use to train your classifiers. You must associate the features found in each row / line with the output found in the corresponding row / line in train-output.csv to create a full case.

test-features.csv contains the features of the examples / cases that you will need to predict using your trained model.

See train.names for header descriptions of the train and test features. This will be helpful in understanding how you may need to normalize, bin, or otherwise update your data to better suit your classifiers.

You can use test-submission.demo to see what an example submission file should look like. This file is a baseline case - you can even upload it to make sure your Kaggle account is working correctly!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import pandas as pd
from sklearn import svm, metrics
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split

from collections import defaultdict
import math
from random import seed
from random import random

%matplotlib inline



Batching notes:
Intial analysis of the training set showed the following information:
from print(traincombined['output'].value_counts())
        print(traincombined.groupby('output').mean())
24720 people below threshold, 7841 above.


As marital status averages were the following, I binned by married vs unmarried due to notably different outputs. 

--marital-status        |||         output                   
 Divorced                    0.104209  
 Married-AF-spouse           0.434783  
 Married-civ-spouse          0.446848  
 Married-spouse-absent       0.081340  
 Never-married               0.045961  
 Separated                   0.064390  
 Widowed                     0.085599  
 
Similarly, for relationship, I binned by Spouse vs not spouse for the same reason as above, and then decided that the relationship column was redunant to marital status, and then deleted it. 

--Relationship      |||  output                    
 Husband              0.448571  
 Not-in-family        0.103070  
 Other-relative       0.037717  
 Own-child            0.013220  
 Unmarried            0.063262  
 Wife                 0.475128 
 
There is a very notable increase in capital gain and loss for people above the income threshold. I decided on a 4 bins for capital gain and capital loss. 

--Averages for 0--
 age 36.783738 
 education-num 9.595065 
 capital-gain 148.752468
 capital-loss 53.142921 
 hours-per-week 38.840210  

--Averages for 1--
 age 44.249841
 education-num 11.611657  
 capital-gain 4006.142456  
 capital-loss 195.001530 
 hours-per-week 45.473026 




In [74]:
class Data:
    def __init__(self,features):
        self.features=features
        
    def preprocess(self):    
        
        #self.features= pd.read_csv("traincombined.csv")
        #self.features =self.features.dropna()
        #https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
        self.features=self.features.drop(columns =
                 ["fnlwgt","race", "sex","native-country",
                  "hours-per-week","workclass", "education-num","occupation","relationship"])
        
        #self.features=self.features.drop(columns = 
          #  ["workclass", "educatio'DataFrame' object has no attribute 'ravel'n","marital-status","occupation",
         #    "relationship","race","sex","native-country"])

        
        #print(self.features['education'].value_counts())
        self.features['education']=np.where(self.features['education'] ==' Preschool',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 1st-4th',
                                                ' Some-school', self.features['education'])       
        self.features['education']=np.where(self.features['education'] ==' 5th-6th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 7th-8th',
                                                ' Some-school', self.features['education'])       
        self.features['education']=np.where(self.features['education'] ==' 9th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 10th',
                                                ' Some-school', self.features['education'])       
        self.features['education']=np.where(self.features['education'] ==' 11th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 12th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Assoc-voc',
                                                ' Assoc-professional', self.features['education'])    
        self.features['education']=np.where(self.features['education'] ==' Prof-school',
                                                ' Assoc-professional', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Assoc-acdm',
                                                ' Assoc-professional', self.features['education'])        
        self.features['education']=np.where(self.features['education'] ==' Some-college',
                                                ' HS-grad or Some-college', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' HS-grad',
                                                ' HS-grad or Some-college', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Masters',
                                                ' Graduate-degree', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Doctorate',
                                                ' Graduate-degree', self.features['education'])
        
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Divorced',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Never-married',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Separated',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Widowed',
                                                ' Not Married', self.features['marital-status'])  
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-spouse-absent',
                                                ' Not Married', self.features['marital-status'])        
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-civ-spouse',
                                                ' Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-AF-spouse',
                                                ' Married', self.features['marital-status'])  
        
        #https://scikit-learn.org/stable/modules/preprocessing.html
        #Normalization of continuous data
        self.features['age'] = preprocessing.scale(self.features['age'])
        self.features['capital-gain'] = preprocessing.scale(self.features['capital-gain'])
        self.features['capital-loss'] = preprocessing.scale(self.features['capital-loss'])
        #print(data_final)
        
        #print(self.features)
        
        #Need to get capital gain and loss working! Make a new collumn?
        
        #self.features['capital-gain']=self.features['capital-gain'].astype(float)

        
        
        
        #print(self.features['capital-gain'].value_counts())
        
        #self.features['capital-gain']=np.where(self.features['capital-gain'].astype(float) < 149,
          #                                      ' Low', self.features['capital-gain'])
        
        #self.features['capital-gain']=np.where((self.features['capital-gain'].astype(float) > 149) &
        #         (self.features['capital-gain'].astype(float) <2077),' Mid-Low', self.features['capital-gain'])        
        #self.features['capital-gain']=np.where((self.features['capital-gain'] > 2077) &
           #      (self.features['capital-gain'] <4006),' Mid-High', self.features['capital-gain']) 
       
        #self.features['capital-gain']=np.where(int(self.features['capital-gain']) > 4006,
        #                                        ' High', self.features['capital-gain'])        

        """
        capital losses: 0, 53.142921, 124.0722255, 195.001530, and up
        
        %matplotlib inline
        pd.crosstab(self.features['age'],self.features['output']).plot(kind='bar')
        plt.title('Purchase Frequency for Job Title')
        plt.xlabel('Job')
        plt.ylabel('Frequency of Purchase')
        plt.savefig('purchase_fre_job')
        """
        
        #print(self.features['relationship'].unique())        
        #print(self.features['education'].value_counts())
        #print(self.features['capital-gain'].value_counts())
        #print(traincombined['output'].value_counts())
        #print(self.features.groupby('marital-status').mean())
        #print(self.features.groupby('age').mean())
        


        #Reference: https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
        #print(self.features['education'].unique()) 
        #print(self.features['marital-status'].unique()) 
        
        
        category_variables=['education','marital-status']
        for var in category_variables:
            cat_list='var'+'_'+var
            cat_list = pd.get_dummies(self.features[var], prefix=var)
            data1=self.features.join(cat_list)
            self.features=data1
            
        category_variables=['education','marital-status']
        self.features_vars=self.features.columns.values.tolist()
        to_keep=[i for i in self.features_vars if i not in category_variables]
            
        data_final=self.features[to_keep]
        data_final.columns.values
        #print(data_final.columns.values)
        #print(data_final)
        
        
        
        return data_final
        
        


In [75]:

#https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a

In [76]:
traindatafeatures= pd.read_csv("train-features_scottannotated.csv")
trainoutput= pd.read_csv("train-output_scottannotated.csv")
testdatafeatures= pd.read_csv("test-features_scottannotated.csv")  

data_train = Data(traindatafeatures).preprocess()
data_test= Data(testdatafeatures).preprocess()


#print(trainoutput)
trainoutputArray=np.asarray(trainoutput)
np.transpose(trainoutputArray)



#data_test.preprocess()
#print(data_train)
#print(trainoutputArray.ravel())

#data_final_vars=data_train.columns.values.tolist()


        #y=['y']
#X=[i for i in data_final_vars if i not in y]
  
logreg = LogisticRegression()
"""        
rfe = RFE(logreg, 5)
rfe = rfe.fit(data_train, trainoutputArray.ravel())
print(rfe.support_)
print(rfe.ranking_)
          
  
        Output for 'age' 'capital-gain' 'capital-loss' 'education_ Assoc-professional'
        'education_ Bachelors' 'education_ Graduate-degree'
        'education_ HS-grad or Some-college' 'education_ Some-school'
        'marital-status_ Married' 'marital-status_ Not Married']
        [False  True False False False False  True  True  True  True]
        [5 1 6 4 3 2 1 1 1 1]

        
        data_final=data_final.drop(columns =
                 ["age",'capital-loss', 'education_ Assoc-professional'
        ,'education_ Bachelors', 'education_ Graduate-degree'])
"""                                               
for epoch in range (3):
        x_train, x_test, y_train, y_test = train_test_split(
             data_train, trainoutputArray.ravel(), test_size=0.25, random_state=5)
        
        logreg.fit(x_train, y_train)
        score = logreg.score(x_test, y_test)
        print("Epoch #:", epoch, " Score: ",score) 
""" 
#To output to a file
predictions= logreg.predict(data_test)
rg=range(16281)
print(predictions)
   
        # dictionary of lists  
dict = {'Category': predictions }  
     
df = pd.DataFrame(dict) 
  
        # saving the dataframe 
df.to_csv('ScottSubmission.csv') 
"""    

ValueError: could not convert string to float: ' United-States'

In [73]:
        
        #train_x,test_x,train_y,test_y= train_test_split(
         #      X,y, test_size=0.25, shuffle=True)